# Important

This notebook is for reproducibility purposes. This is to help with cleaning the data.


In the data folder there is a clean example folder. This folder houses the CSVs that I manually cleaned. This notebook will show you the steps of the data cleaning process and justifications.

In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Load Datasets

# ICU Datasets
d_items=pd.read_csv('../data/icu/d_items.csv')
# Provide addition context for resources used
icu_stays=pd.read_csv('../data/icu/icustays.csv')
# ICU Stay data
ingredient_events=pd.read_csv('../data/icu/ingredientevents.csv')
# provides context for ingredients used
input_events=pd.read_csv('../data/icu/inputevents.csv')
# provides input events context
output_events=pd.read_csv('../data/icu/outputevents.csv')
# Provides output events context
procedure_events=pd.read_csv('../data/icu/procedureevents.csv')
# Provides context for procedure events



# Hospital Datasets

admissions=pd.read_csv('../data/hosp/admissions.csv')
# Admission data
hcpcs_code=pd.read_csv('../data/hosp/d_hcpcs.csv')
# Hospital procedure code
diagnoses_code=pd.read_csv('../data/hosp/d_icd_diagnoses.csv')
# Diagnoses codes
d_procedure_code=pd.read_csv('../data/hosp/d_icd_procedures.csv')
# Procedure Codes
labitems=pd.read_csv('../data/hosp/d_labitems.csv')
# Lab item description
diagnoses=pd.read_csv(r"../data/hosp/diagnoses_icd.csv.gz")
# Patient diagnoses data
drgcodes=pd.read_csv('../data/hosp/drgcodes.csv')
# Drug codes
labevents=pd.read_csv('../data/hosp/labevents.csv')
# Lab events
omr=pd.read_csv('../data/hosp/omr.csv')
# OMR
h_output_events=pd.read_csv('../data/hosp/outputevents.csv')
#output events
patients=pd.read_csv('../data/hosp/patients.csv')
# Patient data
prescriptions=pd.read_csv('../data/hosp/prescriptions.csv')
#Prescription
procedure_code=pd.read_csv(r"../data/hosp/procedures_icd.csv.gz")
# Procedure code
services=pd.read_csv('../data/hosp/services.csv')
# Services
transfers=pd.read_csv('../data/hosp/transfers.csv')
# Transfers


# Data sets not used

Chart_events: Too granular: notates everything that happened regarding a patient. 

Date Time events: Doesn't align with the granularity of the project. Too time specific. The time it takes to create and store a chart has little value based on EDA.

Ingredients Event: Dropping doesn't align with the goal

### Hopsital

EMAR & Emar details : Redundant dataset. Doesn't offer any unique data that I can use. Can get medication from prescription. Not too worried with chart time
Microbiology events: Note something that we can control with lab procedures. Outside the scope

Pharmacy: Doesn't align with granularity

POE & POE Detail: Not using the order which Physicians entered data. So POE detail is dropped along side POE.

Hospital procedure code: Doesn' join into my data sets. Using other hospital codes




# Data Preparation

## ICU Data

In [3]:
d_items

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,226228,Gender,Gender,chartevents,ADT,NaN,Text,NaN,NaN
1,226545,Race,Race,chartevents,ADT,NaN,Text,NaN,NaN
2,229877,Suction events (CH),Suction events (CH),chartevents,ECMO,NaN,Text,NaN,NaN
3,229875,Oxygenator visible (CH),Oxygenator visible (CH),chartevents,ECMO,NaN,Text,NaN,NaN
4,229266,Cannula sites visually inspected (ECMO),Cannula sites visually inspected (ECMO),chartevents,ECMO,NaN,Text,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4009,227452,Tobramycin (Trough),Tobramycin (Trough),chartevents,Labs,NaN,Numeric with tag,NaN,NaN
4010,227451,Tobramycin (Random),Tobramycin (Random),chartevents,Labs,NaN,Numeric with tag,NaN,NaN
4011,227457,Platelet Count,Platelet Count,chartevents,Labs,NaN,Numeric with tag,NaN,NaN
4012,227468,Fibrinogen,Fibrinogen,chartevents,Labs,NaN,Numeric with tag,NaN,NaN


In [4]:
d_items=d_items.drop(columns=['abbreviation','linksto', 'unitname','param_type','lownormalvalue','highnormalvalue'])

In [5]:
d_items

,itemid,label,category
0,226228,Gender,ADT
1,226545,Race,ADT
2,229877,Suction events (CH),ECMO
3,229875,Oxygenator visible (CH),ECMO
4,229266,Cannula sites visually inspected (ECMO),ECMO
...,...,...,...
4009,227452,Tobramycin (Trough),Labs
4010,227451,Tobramycin (Random),Labs
4011,227457,Platelet Count,Labs
4012,227468,Fibrinogen,Labs


In [6]:
icu_stays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,in_time,out_time,los
0,10018328,23786647,31269608,Neuro Stepdown,Neuro Stepdown,4/24/2154 23:03,5/2/2154 15:55,7.702512
1,10020187,24104168,37509585,Neuro Surgical Intensive Care Unit (Neuro SICU),Neuro Stepdown,1/15/2169 4:56,1/20/2169 15:47,5.452662
2,10020187,26842957,32554129,Neuro Intermediate,Neuro Intermediate,2/24/2170 18:18,2/25/2170 15:15,0.872685
3,10012853,27882036,31338022,Trauma SICU (TSICU),Trauma SICU (TSICU),11/26/2176 2:34,11/29/2176 20:58,3.766725
4,10020740,25826145,32145159,Trauma SICU (TSICU),Trauma SICU (TSICU),6/3/2150 20:12,6/4/2150 21:05,1.037106
...,...,...,...,...,...,...,...,...
135,10020786,23488445,33683112,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),6/9/2189 12:46,6/10/2189 22:58,1.424757
136,10020740,23831430,35026312,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),3/11/2150 15:34,3/19/2150 2:17,7.446424
137,10032725,20611640,30101877,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),3/22/2143 6:42,3/25/2143 15:05,3.349687
138,10037928,24656677,39804682,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),12/21/2178 6:05,12/22/2178 2:16,0.840856


In [7]:
icu_stays=icu_stays.drop(columns=['hadm_id','last_careunit', 'in_time','out_time'])

# In time and out time are dropped due to redunancy. They are used to form LOS (Length of stay)

In [8]:
icu_stays['subject_id'].nunique()

100

In [9]:
icu_stays['first_careunit']= icu_stays['first_careunit'].str.strip().str.lower()

#normalize first_careunit data

In [10]:
icu_stays.isna().sum()

# There are zero missing in the data set

# First and Last care units are keep to see if is an issue between transporting to care units

subject_id        0
stay_id           0
first_careunit    0
los               0
dtype: int64

In [11]:
icu_stays

,subject_id,stay_id,first_careunit,los
0,10018328,31269608,neuro stepdown,7.702512
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662
2,10020187,32554129,neuro intermediate,0.872685
3,10012853,31338022,trauma sicu (tsicu),3.766725
4,10020740,32145159,trauma sicu (tsicu),1.037106
...,...,...,...,...
135,10020786,33683112,medical/surgical intensive care unit (micu/sicu),1.424757
136,10020740,35026312,medical/surgical intensive care unit (micu/sicu),7.446424
137,10032725,30101877,medical/surgical intensive care unit (micu/sicu),3.349687
138,10037928,39804682,medical/surgical intensive care unit (micu/sicu),0.840856


In [12]:
input_events

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10005817,20626031,32604416,4793,2132-12-16 19:50:00,2132-12-16 19:51:00,2132-12-16 19:50:00,225798,1.000000,dose,...,Main order parameter,Drug Push,91.0,500.0,ml,0,0,FinishedRunning,1.000000,1.000000
1,10005817,20626031,32604416,92805,2132-12-15 20:15:00,2132-12-15 20:16:00,2132-12-15 20:11:00,225798,1.000000,dose,...,Main order parameter,Drug Push,91.0,500.0,ml,0,0,FinishedRunning,1.000000,1.000000
2,10005817,20626031,32604416,20310,2132-12-17 09:15:00,2132-12-17 09:16:00,2132-12-17 09:28:00,225798,1.000000,dose,...,Main order parameter,Drug Push,91.0,500.0,ml,0,0,FinishedRunning,1.000000,1.000000
3,10005817,20626031,32604416,79166,2132-12-16 09:36:00,2132-12-16 09:37:00,2132-12-16 09:37:00,225798,1.000000,dose,...,Main order parameter,Drug Push,91.0,500.0,ml,0,0,FinishedRunning,1.000000,1.000000
4,10005817,20626031,32604416,92805,2132-12-15 20:10:00,2132-12-15 21:10:00,2132-12-15 20:10:00,221456,2.000000,grams,...,Additives ...,Continuous IV,91.0,100.0,ml,0,0,FinishedRunning,2.000000,0.033333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20399,10019003,29279905,34107647,88156,2153-03-29 20:58:00,2153-03-29 20:59:00,2153-03-29 20:58:00,221385,0.500000,mg,...,Main order parameter,Drug Push,96.0,NaN,NaN,0,0,FinishedRunning,0.500000,0.500000
20400,10019003,29279905,34107647,83144,2153-03-30 00:00:00,2153-03-30 00:01:00,2153-03-30 01:24:00,221385,0.500000,mg,...,Main order parameter,Drug Push,96.0,NaN,NaN,0,0,FinishedRunning,0.500000,0.500000
20401,10019003,29279905,34107647,68979,2153-03-28 02:57:00,2153-03-28 04:48:00,2153-03-28 02:58:00,221906,0.319770,mg,...,Main order parameter,Continuous Med,96.0,250.0,ml,0,0,ChangeDose/Rate,8.000000,0.030000
20402,10019003,29279905,34107647,68979,2153-03-28 06:05:00,2153-03-28 08:00:00,2153-03-28 06:11:00,221906,0.110377,mg,...,Main order parameter,Continuous Med,96.0,250.0,ml,0,0,Paused,7.532536,0.010000


In [13]:
input_events.isna().sum()

subject_id                          0
hadm_id                             0
stay_id                             0
caregiver_id                        0
starttime                           0
endtime                             0
storetime                           0
itemid                              0
amount                              0
amountuom                           0
rate                             9366
rateuom                          9366
orderid                             0
linkorderid                         0
ordercategoryname                   0
secondaryordercategoryname       6260
ordercomponenttypedescription       0
ordercategorydescription            0
patientweight                       0
totalamount                      3314
totalamountuom                   3312
isopenbag                           0
continueinnextdept                  0
statusdescription                   0
originalamount                      0
originalrate                        0
dtype: int64

In [14]:
input_events=input_events.drop(columns=['hadm_id','caregiver_id','starttime','endtime','storetime','amount','amountuom','rate',
                                        'rateuom', 'orderid','linkorderid','ordercategoryname','secondaryordercategoryname',
                                        'patientweight','totalamount','totalamountuom','isopenbag','continueinnextdept',
                                        'originalamount','originalrate','statusdescription'])

In [15]:
input_events

,subject_id,stay_id,itemid,ordercomponenttypedescription,ordercategorydescription
0,10005817,32604416,225798,Main order parameter,Drug Push
1,10005817,32604416,225798,Main order parameter,Drug Push
2,10005817,32604416,225798,Main order parameter,Drug Push
3,10005817,32604416,225798,Main order parameter,Drug Push
4,10005817,32604416,221456,Additives ...,Continuous IV
...,...,...,...,...,...
20399,10019003,34107647,221385,Main order parameter,Drug Push
20400,10019003,34107647,221385,Main order parameter,Drug Push
20401,10019003,34107647,221906,Main order parameter,Continuous Med
20402,10019003,34107647,221906,Main order parameter,Continuous Med


In [16]:
input_events=input_events.drop_duplicates()

In [17]:
input_events

,subject_id,stay_id,itemid,ordercomponenttypedescription,ordercategorydescription
0,10005817,32604416,225798,Main order parameter,Drug Push
4,10005817,32604416,221456,Additives ...,Continuous IV
8,10005817,32604416,220949,Main order parameter,Continuous IV
9,10005817,32604416,220949,Mixed solution,Continuous Med
10,10005817,32604416,220949,Mixed solution,Drug Push
...,...,...,...,...,...
20389,10019003,34107647,225975,Main order parameter,Drug Push
20392,10019003,34107647,227522,Main order parameter,Continuous IV
20396,10019003,34107647,227523,Main order parameter,Continuous IV
20399,10019003,34107647,221385,Main order parameter,Drug Push


In [18]:
input_events['ordercategorydescription']= input_events['ordercategorydescription'].str.strip().str.lower()
input_events['ordercomponenttypedescription']= input_events['ordercomponenttypedescription'].str.strip().str.lower()

#normalizing variables

C:\Users\tayle\AppData\Local\Temp\ipykernel_3960\1076743687.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_events['ordercategorydescription']= input_events['ordercategorydescription'].str.strip().str.lower()
C:\Users\tayle\AppData\Local\Temp\ipykernel_3960\1076743687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_events['ordercomponenttypedescription']= input_events['ordercomponenttypedescription'].str.strip().str.lower()


In [19]:
#aggregating variables by subject_id

inputs_agg = input_events.groupby('subject_id').agg({
    'ordercomponenttypedescription': lambda x: list(set(x)),
    'ordercategorydescription': lambda x: list(set(x))
}).reset_index()

In [20]:
inputs_agg

,subject_id,ordercomponenttypedescription,ordercategorydescription
0,10000032,[main order parameter],"[continuous iv, bolus]"
1,10001217,[additives ...,"[drug push, continuous iv, bolus]"
2,10001725,[additives ...,"[drug push, continuous iv, bolus]"
3,10002428,[additives ...,"[bolus, continuous med, continuous iv, non iv ..."
4,10002495,[additives ...,"[bolus, continuous med, continuous iv, non iv ..."
...,...,...,...
94,10038999,[additives ...,"[continuous med, drug push, continuous iv, bolus]"
95,10039708,[additives ...,"[continuous med, drug push, continuous iv, bolus]"
96,10039831,[additives ...,"[continuous med, drug push, continuous iv, bolus]"
97,10039997,"[main order parameter, additives ...","[continuous med, drug push, continuous iv, bolus]"


In [21]:
#Reason behind the drops.

#Caregiver is a unique Identifer and doesnt add any value

#Start end and store time dont align with granularity

# Rate and Rateuom had over 45% missing values. I am also more concerned about the dose they received rather than they
# rate the received their prescription

# Orderid and linkedorder are uniqueids

# Secondaryordercategoryname is additon classifier that is redundany. The main category suffices for the ordercategory name

# Original Amount and Original Rate are dropped because tehy aren't of main focus. Mainly want to see trends between whats issued
# and how that affects patients 

In [22]:
output_events

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valueuom
0,10002428,23473524,35479615,29441,2156-05-15 18:00:00,2156-05-15 17:42:00,226583,600,ml
1,10002428,23473524,35479615,29441,2156-05-15 12:00:00,2156-05-15 12:08:00,226559,60,ml
2,10002428,23473524,35479615,29441,2156-05-15 13:00:00,2156-05-15 13:00:00,226559,45,ml
3,10002428,23473524,35479615,29441,2156-05-15 08:00:00,2156-05-15 08:39:00,226559,125,ml
4,10002428,23473524,35479615,29441,2156-05-15 14:00:00,2156-05-15 13:56:00,226559,60,ml
...,...,...,...,...,...,...,...,...,...
9357,10016742,29281842,37057036,82943,2178-07-07 11:00:00,2178-07-07 13:25:00,226559,230,ml
9358,10016742,29281842,37057036,82943,2178-07-07 15:00:00,2178-07-07 17:40:00,226559,120,ml
9359,10016742,29281842,37057036,82943,2178-07-07 09:00:00,2178-07-07 09:25:00,226559,250,ml
9360,10016742,29281842,37057036,82943,2178-07-07 18:00:00,2178-07-07 18:18:00,226559,80,ml


In [23]:
output_events.isna().sum()

subject_id      0
hadm_id         0
stay_id         0
caregiver_id    0
charttime       0
storetime       0
itemid          0
value           0
valueuom        0
dtype: int64

In [24]:
output_events=output_events.drop(columns=['hadm_id','caregiver_id', 'charttime','storetime'])

# Caregiver is being removed because its a uniqueid
# Not using the times. They don't align with my granularity

In [25]:
output_events

,subject_id,stay_id,itemid,value,valueuom
0,10002428,35479615,226583,600,ml
1,10002428,35479615,226559,60,ml
2,10002428,35479615,226559,45,ml
3,10002428,35479615,226559,125,ml
4,10002428,35479615,226559,60,ml
...,...,...,...,...,...
9357,10016742,37057036,226559,230,ml
9358,10016742,37057036,226559,120,ml
9359,10016742,37057036,226559,250,ml
9360,10016742,37057036,226559,80,ml


In [26]:
procedure_events.isna().sum()

subject_id                     0
hadm_id                        0
stay_id                        0
caregiver_id                 242
starttime                      0
endtime                        0
storetime                      0
itemid                         0
value                          0
valueuom                     771
location                    1115
locationcategory            1115
orderid                        0
linkorderid                    0
ordercategoryname              0
ordercategorydescription       0
patientweight                  0
isopenbag                      0
continueinnextdept             0
statusdescription              0
ORIGINALAMOUNT                 0
ORIGINALRATE                   0
dtype: int64

In [27]:
procedure_events['location'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1468 entries, 0 to 1467
Series name: location
Non-Null Count  Dtype 
--------------  ----- 
353 non-null    object
dtypes: object(1)
memory usage: 11.6+ KB


In [28]:
procedure_events=procedure_events.drop(columns=['hadm_id','caregiver_id', 'starttime','endtime',
                                                'storetime','value','valueuom','location','locationcategory','orderid','linkorderid',
                                               'patientweight','isopenbag','continueinnextdept','statusdescription','ORIGINALAMOUNT',
                                               'ORIGINALRATE'])

In [29]:
procedure_events['ordercategoryname']= procedure_events['ordercategoryname'].str.strip().str.lower()
procedure_events['ordercategorydescription']= procedure_events['ordercategorydescription'].str.strip().str.lower()


#normalizing variables

In [30]:
proc_agg = procedure_events.groupby('subject_id').agg({
    'ordercategoryname': lambda x: list(set(x)),
    'ordercategorydescription': lambda x: list(set(x))
}).reset_index()

In [31]:
proc_agg

,subject_id,ordercategoryname,ordercategorydescription
0,10000032,"[peripheral lines, procedures]","[task, continuousprocess]"
1,10001217,"[imaging, invasive lines, peripheral lines, co...","[task, continuousprocess]"
2,10001725,[peripheral lines],[continuousprocess]
3,10002428,"[communication, ventilation, imaging, intubati...","[task, continuousprocess]"
4,10002495,"[communication, ventilation, imaging, tubes, p...","[task, continuousprocess]"
...,...,...,...
95,10038999,"[communication, ventilation, imaging, tubes, i...","[task, continuousprocess]"
96,10039708,"[ventilation, imaging, tubes, intubation/extub...","[task, continuousprocess]"
97,10039831,"[imaging, invasive lines, peripheral lines, pr...","[task, continuousprocess]"
98,10039997,"[imaging, invasive lines, peripheral lines]","[task, continuousprocess]"


In [32]:
#Dropping caregiver id because its a uniqueid
# Dropping times as they doing align with my granularity
# dropping location and location category as over 75% of the data is missing.

## Hospital

In [33]:
admissions

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10004235,24181354,2196-02-24 14:38:00,2196-03-04 14:02:00,NaN,URGENT,P03YMR,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicaid,ENGLISH,SINGLE,BLACK/CAPE VERDEAN,2196-02-24 12:15:00,2196-02-24 17:07:00,0
1,10009628,25926192,2153-09-17 17:08:00,2153-09-25 13:20:00,NaN,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,?,MARRIED,HISPANIC/LATINO - PUERTO RICAN,NaN,NaN,0
2,10018081,23983182,2134-08-18 02:02:00,2134-08-23 19:35:00,NaN,URGENT,P233F6,TRANSFER FROM HOSPITAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2134-08-17 16:24:00,2134-08-18 03:15:00,0
3,10006053,22942076,2111-11-13 23:39:00,2111-11-15 17:20:00,2111-11-15 17:20:00,URGENT,P38TI6,TRANSFER FROM HOSPITAL,DIED,Medicaid,ENGLISH,NaN,UNKNOWN,NaN,NaN,1
4,10031404,21606243,2113-08-04 18:46:00,2113-08-06 20:57:00,NaN,URGENT,P07HDB,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,10038992,24745425,2187-07-29 01:05:00,2187-08-03 17:02:00,NaN,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
271,10008287,22168393,2145-09-28 01:17:00,2145-10-02 13:35:00,NaN,SURGICAL SAME DAY ADMISSION,P898NM,PHYSICIAN REFERRAL,HOME HEALTH CARE,Other,ENGLISH,SINGLE,WHITE,NaN,NaN,0
272,10022880,27708593,2177-03-12 07:15:00,2177-03-19 14:25:00,NaN,SURGICAL SAME DAY ADMISSION,P99698,PHYSICIAN REFERRAL,HOME,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
273,10004457,23251352,2141-12-17 11:00:00,2141-12-21 15:56:00,NaN,SURGICAL SAME DAY ADMISSION,P41R5N,PHYSICIAN REFERRAL,REHAB,Medicare,ENGLISH,SINGLE,OTHER,NaN,NaN,0


In [34]:
admissions=admissions.drop(columns=['hadm_id','admittime','dischtime','deathtime','admission_type','admit_provider_id','admission_location',
                                   'discharge_location','edregtime','edouttime','hospital_expire_flag'])

In [35]:
admissions.isna().sum()

subject_id         0
insurance          0
language           0
marital_status    12
race               0
dtype: int64

In [36]:
admissions['insurance']= admissions['insurance'].str.strip().str.lower()
admissions['language']= admissions['language'].str.strip().str.lower()
admissions['marital_status']= admissions['marital_status'].str.strip().str.lower()
admissions['race']= admissions['race'].str.strip().str.lower()

#normalizing variables

In [37]:
adm_agg=admissions.groupby('subject_id').agg({
    'insurance': lambda x: list(set(x)),
    'language': lambda x: list(set(x)),
    'marital_status': lambda x: list(set(x)),
    'race': lambda x: list(set(x))
}).reset_index()

In [38]:
adm_agg

,subject_id,insurance,language,marital_status,race
0,10000032,"[other, medicaid]",[english],[widowed],[white]
1,10001217,[other],[?],[married],[white]
2,10001725,[other],[english],[married],[white]
3,10002428,[medicare],[english],[widowed],[white]
4,10002495,[medicare],[english],[married],[unknown]
...,...,...,...,...,...
95,10038999,[medicare],[english],[single],[white]
96,10039708,[other],[english],[single],[black/african american]
97,10039831,[other],[english],[married],[unable to obtain]
98,10039997,[medicare],[english],"[married, widowed]",[black/african american]


In [39]:
# Admittime, dischtime, edregtime, edouttime. Don't align with my scope removing time
# admission location is out of our control so its being removed
# Discharge location: Patients should be healthy enough to where they don't die if they are being discharged
# Admit Provider ID provides no information. Its a unique id

In [40]:
diagnoses_code

,icd_code,icd_version,long_title
0,0090,9,"Infectious colitis, enteritis, and gastroenter..."
1,01160,9,"Tuberculous pneumonia [any form], unspecified"
2,01186,9,"Other specified pulmonary tuberculosis, tuberc..."
3,01200,9,"Tuberculous pleurisy, unspecified"
4,01236,9,"Tuberculous laryngitis, tubercle bacilli not f..."
...,...,...,...
109770,Z88,10,"Allergy status to drugs, medicaments and biolo..."
109771,Z89012,10,Acquired absence of left thumb
109772,Z90410,10,Acquired total absence of pancreas
109773,Z948,10,Other transplanted organ and tissue status


In [41]:
diagnoses_code['long_title']= diagnoses_code['long_title'].str.strip().str.lower()

In [42]:
diagnoses_code.isna().sum()

#its worth considering dropping all the data that uses version 9 as its outdate

icd_code       0
icd_version    0
long_title     0
dtype: int64

In [43]:
d_procedure_code

,icd_code,icd_version,long_title
0,0039,9,Other computer assisted surgery
1,0048,9,Insertion of four or more vascular stents
2,0074,9,"Hip bearing surface, metal-on-polyethylene"
3,0077,9,"Hip bearing surface, ceramic-on-polyethylene"
4,0126,9,Insertion of catheter(s) into cranial cavity o...
...,...,...,...
85252,F15Z68Z,10,Computerized Dynamic Posturography Assessment ...
85253,HZ37ZZZ,10,Individual Counseling for Substance Abuse Trea...
85254,X27L395,10,Dilation of Proximal Left Popliteal Artery wit...
85255,X2C0361,10,"Extirpation of Matter from Coronary Artery, On..."


In [44]:
d_procedure_code['long_title']= d_procedure_code['long_title'].str.strip().str.lower()

In [45]:
d_procedure_code.isna().sum()

icd_code       0
icd_version    0
long_title     0
dtype: int64

In [46]:
labitems

,itemid,label,fluid,category
0,50808,Free Calcium,Blood,Blood Gas
1,50826,Tidal Volume,Blood,Blood Gas
2,50813,Lactate,Blood,Blood Gas
3,52029,% Ionized Calcium,Blood,Blood Gas
4,50801,Alveolar-arterial Gradient,Blood,Blood Gas
...,...,...,...,...
1617,52249,Delete,Cerebrospinal Fluid,Hematology
1618,52251,Delete,Cerebrospinal Fluid,Hematology
1619,52266,Macrophage,Cerebrospinal Fluid,Hematology
1620,52262,Immunophenotyping,Cerebrospinal Fluid,Hematology


In [47]:
labitems.isna().sum()

itemid      0
label       3
fluid       0
category    0
dtype: int64

In [48]:
# 3 missings is minimal. Will keep the data description as is

In [49]:
diagnoses

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10035185,22580999,3,4139,9
1,10035185,22580999,10,V707,9
2,10035185,22580999,1,41401,9
3,10035185,22580999,9,3899,9
4,10035185,22580999,11,V8532,9
...,...,...,...,...,...
4501,10004733,27411876,19,3129,9
4502,10004733,27411876,30,30000,9
4503,10004733,27411876,26,4739,9
4504,10004733,27411876,22,56210,9


In [50]:
diagnoses=diagnoses.drop(columns=['hadm_id','seq_num'])

In [51]:
# Not too concern with the order of the diagnoses so I am dropping seq_number.

In [52]:
drgcodes

,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,10004235,22187210,HCFA,864,FEVER,NaN,NaN
1,10026255,22059910,HCFA,180,RESPIRATORY NEOPLASMS W MCC,NaN,NaN
2,10032725,20611640,HCFA,54,NERVOUS SYSTEM NEOPLASMS W MCC,NaN,NaN
3,10005866,21636229,HCFA,393,OTHER DIGESTIVE SYSTEM DIAGNOSES W MCC,NaN,NaN
4,10008454,20291550,HCFA,956,"LIMB REATTACHMENT, HIP & FEMUR PROC FOR MULTIP...",NaN,NaN
...,...,...,...,...,...,...,...
449,10040025,27259207,HCFA,982,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,NaN,NaN
450,10001725,25563031,HCFA,982,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,NaN,NaN
451,10023239,21759936,HCFA,987,NON-EXTENSIVE O.R. PROC UNRELATED TO PRINCIPAL...,NaN,NaN
452,10019003,26529390,HCFA,987,NON-EXTENSIVE O.R. PROC UNRELATED TO PRINCIPAL...,NaN,NaN


In [53]:
drgcodes['drg_severity'] = drgcodes['drg_severity'].astype('object')
drgcodes['drg_mortality'] = drgcodes['drg_mortality'].astype('object')


In [54]:
severity_map={1:'mild',
              2:'moderate',
              3:'severe',
             4:'critical'}

drgcodes.loc[drgcodes['drg_severity'].notna(),'drg_severity']=drgcodes['drg_severity'].map(severity_map)

In [55]:
mortality_risk_map={
              1:'minor',
              2:'moderate',
              3:'major',
             4:'extreme'}

drgcodes.loc[drgcodes['drg_mortality'].notna(),'drg_mortality']=drgcodes['drg_mortality'].map(mortality_risk_map)

In [56]:
drgcodes

,subject_id,hadm_id,drg_type,drg_code,description,drg_severity,drg_mortality
0,10004235,22187210,HCFA,864,FEVER,NaN,NaN
1,10026255,22059910,HCFA,180,RESPIRATORY NEOPLASMS W MCC,NaN,NaN
2,10032725,20611640,HCFA,54,NERVOUS SYSTEM NEOPLASMS W MCC,NaN,NaN
3,10005866,21636229,HCFA,393,OTHER DIGESTIVE SYSTEM DIAGNOSES W MCC,NaN,NaN
4,10008454,20291550,HCFA,956,"LIMB REATTACHMENT, HIP & FEMUR PROC FOR MULTIP...",NaN,NaN
...,...,...,...,...,...,...,...
449,10040025,27259207,HCFA,982,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,NaN,NaN
450,10001725,25563031,HCFA,982,EXTENSIVE O.R. PROCEDURE UNRELATED TO PRINCIPA...,NaN,NaN
451,10023239,21759936,HCFA,987,NON-EXTENSIVE O.R. PROC UNRELATED TO PRINCIPAL...,NaN,NaN
452,10019003,26529390,HCFA,987,NON-EXTENSIVE O.R. PROC UNRELATED TO PRINCIPAL...,NaN,NaN


In [57]:
drgcodes=drgcodes.drop(columns=['hadm_id'])

In [58]:
drgcodes.isna().sum()

subject_id         0
drg_type           0
drg_code           0
description        0
drg_severity     233
drg_mortality    233
dtype: int64

In [59]:
drgcodes['drg_mortality'].unique()

array([nan, 'minor', 'moderate', 'major', 'extreme'], dtype=object)

In [60]:
# Diagnosis-related group. drg severity and moratlity changed to object. INstead of 1-4 its minor, to extreme etc. Easier to understand

In [61]:
labevents

,labevent_id,subject_id,hadm_id,specimen_id,itemid,order_provider_id,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,172061,10014354,29600294.0,1808066,51277,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,15.4,15.40,%,10.5,15.5,NaN,ROUTINE,NaN
1,172062,10014354,29600294.0,1808066,51279,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,3.35,3.35,m/uL,4.6,6.1,abnormal,ROUTINE,NaN
2,172068,10014354,29600294.0,1808066,52172,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,49.7,49.70,fL,35.1,46.3,abnormal,ROUTINE,NaN
3,172063,10014354,29600294.0,1808066,51301,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,20.3,20.30,K/uL,4.0,10.0,abnormal,ROUTINE,NaN
4,172050,10014354,29600294.0,1808066,51249,NaN,2148-08-16 00:00:00,2148-08-16 01:30:00,31.1,31.10,g/dL,32.0,37.0,abnormal,ROUTINE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107722,254700,10021487,28998349.0,78791160,50804,NaN,2116-12-07 18:59:00,2116-12-07 19:00:00,35,35.00,mEq/L,21.0,30.0,abnormal,NaN,NaN
107723,254702,10021487,28998349.0,78791160,50818,NaN,2116-12-07 18:59:00,2116-12-07 19:00:00,56,56.00,mm Hg,35.0,45.0,abnormal,NaN,NaN
107724,254707,10021487,28998349.0,78791160,52033,NaN,2116-12-07 18:59:00,2116-12-07 18:59:00,ART.,NaN,NaN,NaN,NaN,NaN,NaN,___
107725,254706,10021487,28998349.0,78791160,50825,NaN,2116-12-07 18:59:00,2116-12-07 18:59:00,39.7,39.70,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
labevents.isna().sum()

labevent_id              0
subject_id               0
hadm_id              28420
specimen_id              0
itemid                   0
order_provider_id    90897
charttime                0
storetime              992
value                 9588
valuenum             12481
valueuom             16203
ref_range_lower      18728
ref_range_upper      18728
flag                 67452
priority              9329
comments             89273
dtype: int64

In [63]:
labevents=labevents.drop(columns=['hadm_id','order_provider_id','charttime','storetime','valuenum','comments'])

In [64]:
labevents

,labevent_id,subject_id,specimen_id,itemid,value,valueuom,ref_range_lower,ref_range_upper,flag,priority
0,172061,10014354,1808066,51277,15.4,%,10.5,15.5,NaN,ROUTINE
1,172062,10014354,1808066,51279,3.35,m/uL,4.6,6.1,abnormal,ROUTINE
2,172068,10014354,1808066,52172,49.7,fL,35.1,46.3,abnormal,ROUTINE
3,172063,10014354,1808066,51301,20.3,K/uL,4.0,10.0,abnormal,ROUTINE
4,172050,10014354,1808066,51249,31.1,g/dL,32.0,37.0,abnormal,ROUTINE
...,...,...,...,...,...,...,...,...,...,...
107722,254700,10021487,78791160,50804,35,mEq/L,21.0,30.0,abnormal,NaN
107723,254702,10021487,78791160,50818,56,mm Hg,35.0,45.0,abnormal,NaN
107724,254707,10021487,78791160,52033,ART.,NaN,NaN,NaN,NaN,NaN
107725,254706,10021487,78791160,50825,39.7,NaN,NaN,NaN,NaN,NaN


In [65]:
# Comments didn't provide valuable insights
#valuenum and value are pretty much the same column. Only difference is how values missing are listed. 
# Time isn't my concerned so removed
# Not too worried who ordered

# Ref_ranges blanks means that there is not a range. 
# Flags being blank is essentially abnormal or normal

In [66]:
omr

,subject_id,chartdate,seq_num,result_name,result_value
0,10011398,2146-12-01,1,Height (Inches),63
1,10011398,2147-01-22,1,Weight (Lbs),127
2,10011398,2146-12-01,1,Weight (Lbs),135
3,10011398,2147-07-24,1,Weight (Lbs),136
4,10011398,2147-03-26,1,Weight (Lbs),136
...,...,...,...,...,...
2959,10021118,2161-11-15,1,Height (Inches),71
2960,10021118,2161-11-15,1,BMI (kg/m2),35.2
2961,10021118,2161-11-29,1,Blood Pressure,111/72
2962,10021118,2161-12-25,1,Blood Pressure,135/76


In [67]:
omr['chartdate']=pd.to_datetime(omr['chartdate'])
omr=omr.sort_values(['subject_id','chartdate'])

In [68]:
# Ivisit_Id Independent visit id. Only used for OMR
omr['ivisit_id']=omr.groupby('subject_id')['chartdate'].rank(method='dense').astype(int)

In [69]:
omr

,subject_id,chartdate,seq_num,result_name,result_value,ivisit_id
1204,10000032,2180-04-27,1,Weight (Lbs),94,1
1241,10000032,2180-04-27,1,Blood Pressure,110/65,1
1203,10000032,2180-05-07,1,Height (Inches),60,2
1207,10000032,2180-05-07,1,BMI (kg/m2),18.0,2
1218,10000032,2180-05-07,17,Weight (Lbs),92.15,2
...,...,...,...,...,...,...
2689,10040025,2147-12-29,1,Weight (Lbs),212,9
2696,10040025,2147-12-29,1,BMI (kg/m2),34.2,9
2692,10040025,2147-12-30,1,BMI (kg/m2),30.3,10
2704,10040025,2147-12-30,1,Weight (Lbs),187.61,10


In [70]:
omr=omr.drop(columns=['chartdate','seq_num'])

In [71]:
omr=omr.pivot_table(
index=['subject_id','ivisit_id'],
columns='result_name',
values='result_value',
aggfunc='last').reset_index()

In [72]:
omr

result_name,subject_id,ivisit_id,BMI (kg/m2),Blood Pressure,Blood Pressure Lying,Blood Pressure Sitting,Blood Pressure Standing (1 min),Blood Pressure Standing (3 mins),Height (Inches),Weight (Lbs)
0,10000032,1,NaN,110/65,NaN,NaN,NaN,NaN,NaN,94
1,10000032,2,18.0,NaN,NaN,NaN,NaN,NaN,60,92.15
2,10000032,3,18.6,106/60,NaN,NaN,NaN,NaN,NaN,95
3,10000032,4,18.7,121/77,NaN,NaN,NaN,NaN,NaN,95.7
4,10000032,5,18.9,100/60,NaN,NaN,NaN,NaN,60,97
...,...,...,...,...,...,...,...,...,...,...
951,10040025,7,30.3,NaN,NaN,NaN,NaN,NaN,NaN,188.00
952,10040025,8,32.3,NaN,NaN,NaN,NaN,NaN,NaN,200.00
953,10040025,9,34.2,NaN,NaN,NaN,NaN,NaN,NaN,212
954,10040025,10,30.3,NaN,NaN,NaN,NaN,NaN,NaN,187.61


In [73]:
omr.isna().sum()

result_name
subject_id                            0
ivisit_id                             0
BMI (kg/m2)                         195
Blood Pressure                      129
Blood Pressure Lying                954
Blood Pressure Sitting              954
Blood Pressure Standing (1 min)     955
Blood Pressure Standing (3 mins)    955
Height (Inches)                     581
Weight (Lbs)                        151
dtype: int64

In [74]:
omr=omr.drop(columns=["Blood Pressure Lying",'Blood Pressure Sitting',"Blood Pressure Standing (1 min)", "Blood Pressure Standing (3 mins)"])

In [75]:
omr

result_name,subject_id,ivisit_id,BMI (kg/m2),Blood Pressure,Height (Inches),Weight (Lbs)
0,10000032,1,NaN,110/65,NaN,94
1,10000032,2,18.0,NaN,60,92.15
2,10000032,3,18.6,106/60,NaN,95
3,10000032,4,18.7,121/77,NaN,95.7
4,10000032,5,18.9,100/60,60,97
...,...,...,...,...,...,...
951,10040025,7,30.3,NaN,NaN,188.00
952,10040025,8,32.3,NaN,NaN,200.00
953,10040025,9,34.2,NaN,NaN,212
954,10040025,10,30.3,NaN,NaN,187.61


In [76]:
omr.isna().sum()

result_name
subject_id           0
ivisit_id            0
BMI (kg/m2)        195
Blood Pressure     129
Height (Inches)    581
Weight (Lbs)       151
dtype: int64

In [77]:
# Back filling height. Weight the age range of the patients being 21+ its highly unlikely that the patients height has changed
# so height data is back filled
omr['Height (Inches)'] = omr.groupby('subject_id')['Height (Inches)'].ffill()


In [78]:
omr.isna().sum()

result_name
subject_id           0
ivisit_id            0
BMI (kg/m2)        195
Blood Pressure     129
Height (Inches)    121
Weight (Lbs)       151
dtype: int64

In [79]:
# Less than 20% data missing for BMI so I data would be kept

In [80]:
h_output_events

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valueuom
0,10002428,23473524,35479615,29441,2156-05-15 18:00:00,2156-05-15 17:42:00,226583,600,ml
1,10002428,23473524,35479615,29441,2156-05-15 12:00:00,2156-05-15 12:08:00,226559,60,ml
2,10002428,23473524,35479615,29441,2156-05-15 13:00:00,2156-05-15 13:00:00,226559,45,ml
3,10002428,23473524,35479615,29441,2156-05-15 08:00:00,2156-05-15 08:39:00,226559,125,ml
4,10002428,23473524,35479615,29441,2156-05-15 14:00:00,2156-05-15 13:56:00,226559,60,ml
...,...,...,...,...,...,...,...,...,...
9357,10016742,29281842,37057036,82943,2178-07-07 11:00:00,2178-07-07 13:25:00,226559,230,ml
9358,10016742,29281842,37057036,82943,2178-07-07 15:00:00,2178-07-07 17:40:00,226559,120,ml
9359,10016742,29281842,37057036,82943,2178-07-07 09:00:00,2178-07-07 09:25:00,226559,250,ml
9360,10016742,29281842,37057036,82943,2178-07-07 18:00:00,2178-07-07 18:18:00,226559,80,ml


In [81]:
h_output_events=h_output_events.drop(columns=['caregiver_id','charttime','storetime'])

In [82]:
h_output_events

,subject_id,hadm_id,stay_id,itemid,value,valueuom
0,10002428,23473524,35479615,226583,600,ml
1,10002428,23473524,35479615,226559,60,ml
2,10002428,23473524,35479615,226559,45,ml
3,10002428,23473524,35479615,226559,125,ml
4,10002428,23473524,35479615,226559,60,ml
...,...,...,...,...,...,...
9357,10016742,29281842,37057036,226559,230,ml
9358,10016742,29281842,37057036,226559,120,ml
9359,10016742,29281842,37057036,226559,250,ml
9360,10016742,29281842,37057036,226559,80,ml


In [83]:
output_events.equals(h_output_events)

False

In [84]:
# Not using h_output_events anymore. Redundant

In [85]:
patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10014729,F,21,2125,2011 - 2013,NaN
1,10003400,F,72,2134,2011 - 2013,2137-09-02
2,10002428,F,80,2155,2011 - 2013,NaN
3,10032725,F,38,2143,2011 - 2013,2143-03-30
4,10027445,F,48,2142,2011 - 2013,2146-02-09
...,...,...,...,...,...,...
95,10004733,M,51,2174,2014 - 2016,NaN
96,10021118,M,62,2161,2014 - 2016,NaN
97,10018501,M,83,2141,2014 - 2016,NaN
98,10007058,M,48,2167,2014 - 2016,NaN


In [86]:
patients.rename(columns={'dod':"death_flag"}, inplace=True)

In [87]:
patients['death_flag']=patients['death_flag'].notna().astype(int)

In [88]:
patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,death_flag
0,10014729,F,21,2125,2011 - 2013,0
1,10003400,F,72,2134,2011 - 2013,1
2,10002428,F,80,2155,2011 - 2013,0
3,10032725,F,38,2143,2011 - 2013,1
4,10027445,F,48,2142,2011 - 2013,1
...,...,...,...,...,...,...
95,10004733,M,51,2174,2014 - 2016,0
96,10021118,M,62,2161,2014 - 2016,0
97,10018501,M,83,2141,2014 - 2016,0
98,10007058,M,48,2167,2014 - 2016,0


In [89]:
patients=patients.drop(columns=['anchor_year'])

In [90]:
patients

,subject_id,gender,anchor_age,anchor_year_group,death_flag
0,10014729,F,21,2011 - 2013,0
1,10003400,F,72,2011 - 2013,1
2,10002428,F,80,2011 - 2013,0
3,10032725,F,38,2011 - 2013,1
4,10027445,F,48,2011 - 2013,1
...,...,...,...,...,...
95,10004733,M,51,2014 - 2016,0
96,10021118,M,62,2014 - 2016,0
97,10018501,M,83,2014 - 2016,0
98,10007058,M,48,2014 - 2016,0


In [91]:
# Anchor year is dropped because the data is aggregated by anchor year group.
# DOD was changed to death flag. This is similar to hospital expired flag. Both indicate if a patient had died or not.
# Keeping death flag together so I wouldn't have to join a third data set to get that additional information

In [92]:
prescriptions

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10027602,28166872,27168639,NaN,NaN,NaN,2201-10-30 12:00:00,NaN,MAIN,Fentanyl Citrate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10027602,28166872,40720238,NaN,NaN,NaN,2201-10-30 12:00:00,NaN,MAIN,Fentanyl Citrate,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10027602,28166872,62845687,NaN,NaN,NaN,2201-10-31 12:00:00,NaN,MAIN,Lorazepam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10027602,28166872,24340150,NaN,NaN,NaN,2201-10-30 12:00:00,NaN,MAIN,Midazolam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10027602,28166872,14435820,NaN,NaN,NaN,2201-10-30 12:00:00,NaN,MAIN,Midazolam,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18082,10038081,20755971,33730068,10038081-378,378.0,P92KOD,2115-10-11 14:00:00,2115-10-13 07:00:00,MAIN,Artificial Tears,...,030016,2.305060e+07,0.4 mL DROPPERETTE,NaN,1-2,DROP,0.1667-0.3333,DRP,NaN,BOTH EYES
18083,10002428,23473524,87358294,10002428-780,780.0,P71IN4,2156-05-12 13:00:00,2156-05-22 18:00:00,MAIN,Artificial Tears,...,030016,2.305060e+07,0.3mL UD,NaN,1-2,DROP,0.1667-0.3333,DRP,NaN,BOTH EYES
18084,10040025,27996267,81941017,10040025-1640,1640.0,P52ORO,2148-01-26 19:00:00,2148-01-26 18:00:00,MAIN,OxyCODONE (Immediate Release),...,046474,9.046446e+08,15mg Tablet,NaN,5-10,mg,0.3333-0.6667,TAB,NaN,PO/NG
18085,10014354,26228185,46019806,10014354-3105,3105.0,P748G6,2150-05-01 01:00:00,2150-05-01 09:00:00,MAIN,Carbamide Peroxide 6.5%,...,008120,7.811207e+10,15mL Bottle,NaN,5-10,DROP,0.3333-0.6667,BTL,1.0,BOTH EARS


In [93]:
prescriptions=prescriptions.drop(columns=['hadm_id','pharmacy_id','poe_id','poe_seq','order_provider_id','starttime','stoptime','gsn','ndc',
                                'prod_strength','form_rx','dose_val_rx','dose_unit_rx','form_val_disp','form_unit_disp','doses_per_24_hrs','route'])

In [94]:
prescriptions

,subject_id,drug_type,drug,formulary_drug_cd
0,10027602,MAIN,Fentanyl Citrate,FENT2I
1,10027602,MAIN,Fentanyl Citrate,FENT2I
2,10027602,MAIN,Lorazepam,LORA2I
3,10027602,MAIN,Midazolam,MIDA2I
4,10027602,MAIN,Midazolam,MIDA2I
...,...,...,...,...
18082,10038081,MAIN,Artificial Tears,REFRESH
18083,10002428,MAIN,Artificial Tears,REFRESH
18084,10040025,MAIN,OxyCODONE (Immediate Release),OXYIR15
18085,10014354,MAIN,Carbamide Peroxide 6.5%,GLYO65OT


In [95]:
prescriptions['drug']= prescriptions['drug'].str.strip().str.lower()

In [96]:
presc_agg = prescriptions.groupby('subject_id').agg({
    'drug': lambda x: list(set(x))
}).reset_index()

In [97]:
# There was a lot of redundant data or data that doesn't align with my granularity so they were removed.

In [98]:
procedure_code

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10011398,27505812,3,2146-12-15,3961,9
1,10011398,27505812,2,2146-12-15,3615,9
2,10011398,27505812,1,2146-12-15,3614,9
3,10014729,23300884,4,2125-03-23,3897,9
4,10014729,23300884,1,2125-03-20,3403,9
...,...,...,...,...,...,...
717,10004733,27411876,3,2174-12-20,4513,9
718,10021118,24490144,4,2161-11-19,5A1221Z,10
719,10021118,24490144,3,2161-11-19,06BP4ZZ,10
720,10021118,24490144,1,2161-11-19,02100Z9,10


In [99]:
procedure_code=procedure_code.drop(columns=['hadm_id','seq_num','chartdate'])

In [100]:
procedure_code

,subject_id,icd_code,icd_version
0,10011398,3961,9
1,10011398,3615,9
2,10011398,3614,9
3,10014729,3897,9
4,10014729,3403,9
...,...,...,...
717,10004733,4513,9
718,10021118,5A1221Z,10
719,10021118,06BP4ZZ,10
720,10021118,02100Z9,10


In [101]:
services

,subject_id,hadm_id,transfertime,prev_service,curr_service
0,10001725,25563031,4/11/2110 15:09,NaN,GYN
1,10019003,28003918,12/21/2148 3:32,NaN,GYN
2,10007818,22987108,6/10/2146 16:38,NaN,MED
3,10004235,24181354,2/24/2196 14:39,NaN,MED
4,10026255,22059910,7/7/2201 18:16,NaN,MED
...,...,...,...,...,...
314,10005909,20199380,10/28/2144 23:20,NaN,VSURG
315,10040025,27996267,1/23/2148 12:18,NaN,VSURG
316,10040025,25933959,12/29/2147 19:37,NaN,VSURG
317,10040025,27259207,12/4/2147 20:49,NaN,VSURG


In [102]:
services=services.drop(columns=['hadm_id','transfertime'])

In [103]:
services["prev_service"] = services["prev_service"].apply(lambda x: None if pd.isna(x) else x)

In [104]:
services

,subject_id,prev_service,curr_service
0,10001725,None,GYN
1,10019003,None,GYN
2,10007818,None,MED
3,10004235,None,MED
4,10026255,None,MED
...,...,...,...
314,10005909,None,VSURG
315,10040025,None,VSURG
316,10040025,None,VSURG
317,10040025,None,VSURG


In [105]:
transfers

,subject_id,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10009049,22995465.0,30030230,discharge,NaN,2174-05-31 14:21:47,NaN
1,10025612,23403708.0,32533329,discharge,NaN,2125-10-03 12:25:27,NaN
2,10020786,23488445.0,37922399,discharge,NaN,2189-06-13 17:25:44,NaN
3,10014078,25809882.0,34694622,discharge,NaN,2166-08-26 14:49:42,NaN
4,10039831,26924951.0,37155928,discharge,NaN,2116-01-02 14:35:02,NaN
...,...,...,...,...,...,...,...
1185,10021118,24490144.0,31983963,transfer,PACU,2161-11-19 07:16:15,2161-11-19 10:04:04
1186,10021118,24490144.0,38367109,transfer,Cardiac Surgery,2161-11-20 21:45:42,2161-11-23 16:06:31
1187,10021118,24490144.0,39362807,transfer,Medicine/Cardiology,2161-11-16 19:53:20,2161-11-19 07:16:15
1188,10021118,24490144.0,38425947,admit,Medicine/Cardiology,2161-11-15 20:10:55,2161-11-16 19:53:20


In [106]:
transfers=transfers.drop(columns=['hadm_id','intime','outtime'])

In [107]:
# not dropping careunit. NAN mean that the patient was discharged and no longer under the hospitals care

# Joining data sets

In [108]:
# Joined Diagnoses
j_diagnoses=diagnoses.merge(diagnoses_code,on=['icd_code','icd_version'],how='inner')

# Joined Procedure Events
j_procedure_code=procedure_code.merge(d_procedure_code,on=['icd_code','icd_version'],how='inner')

In [109]:
j_diagnoses

,subject_id,icd_code,icd_version,long_title
0,10035185,4139,9,other and unspecified angina pectoris
1,10005348,4139,9,other and unspecified angina pectoris
2,10035185,V707,9,examination of participant in clinical trial
3,10012552,V707,9,examination of participant in clinical trial
4,10005817,V707,9,examination of participant in clinical trial
...,...,...,...,...
4501,10004733,E9383,9,intravenous anesthetics causing adverse effect...
4502,10004733,34290,9,"hemiplegia, unspecified, affecting unspecified..."
4503,10004733,40300,9,"hypertensive chronic kidney disease, malignant..."
4504,10004733,0416,9,proteus (mirabilis) (morganii) infection in co...


In [110]:
j_diagnoses=j_diagnoses.drop(columns=['icd_code','icd_version'])

In [111]:
diag_agg = j_diagnoses.groupby('subject_id').agg({
    'long_title': lambda x: list(set(x))
}).reset_index()

In [112]:
diag_agg

,subject_id,long_title
0,10000032,"[diarrhea, tobacco use disorder, other iatroge..."
1,10001217,"[intracranial abscess, pulmonary collapse, str..."
2,10001725,"[vaginal enterocele, congenital or acquired, m..."
3,10002428,[age-related osteoporosis with current patholo...
4,10002495,"[unspecified injury of urethra, initial encoun..."
...,...,...
95,10038999,[unspecified place in other specified resident...
96,10039708,"[hyperosmolality and hypernatremia, hypotensio..."
97,10039831,"[accidental cut, puncture, perforation or hemo..."
98,10039997,"[hemiplegia, unspecified affecting right domin..."


In [113]:
diag_agg.rename(columns={'long_title':'diagnoses_name'}, inplace=True)

In [114]:
diag_exploded=diag_agg.explode('diagnoses_name')

diagnosis_counts = (
    diag_exploded['diagnoses_name']
        .str.lower()                     # normalize text
        .value_counts()                  # count occurrences
        .reset_index()
        .rename(columns={'index': 'diagnoses_name', 'diagnoses_name': 'count'})
)

In [115]:
j_procedure_code

,subject_id,icd_code,icd_version,long_title
0,10011398,3961,9,extracorporeal circulation auxiliary to open h...
1,10014729,3961,9,extracorporeal circulation auxiliary to open h...
2,10009628,3961,9,extracorporeal circulation auxiliary to open h...
3,10005817,3961,9,extracorporeal circulation auxiliary to open h...
4,10023771,3961,9,extracorporeal circulation auxiliary to open h...
...,...,...,...,...
717,10025463,0096,9,infusion of 4-factor prothrombin complex conce...
718,10002428,9390,9,non-invasive mechanical ventilation
719,10004733,3322,9,fiber-optic bronchoscopy
720,10004733,5523,9,closed [percutaneous] [needle] biopsy of kidney


In [116]:
proc_agg

,subject_id,ordercategoryname,ordercategorydescription
0,10000032,"[peripheral lines, procedures]","[task, continuousprocess]"
1,10001217,"[imaging, invasive lines, peripheral lines, co...","[task, continuousprocess]"
2,10001725,[peripheral lines],[continuousprocess]
3,10002428,"[communication, ventilation, imaging, intubati...","[task, continuousprocess]"
4,10002495,"[communication, ventilation, imaging, tubes, p...","[task, continuousprocess]"
...,...,...,...
95,10038999,"[communication, ventilation, imaging, tubes, i...","[task, continuousprocess]"
96,10039708,"[ventilation, imaging, tubes, intubation/extub...","[task, continuousprocess]"
97,10039831,"[imaging, invasive lines, peripheral lines, pr...","[task, continuousprocess]"
98,10039997,"[imaging, invasive lines, peripheral lines]","[task, continuousprocess]"


In [117]:
j_procedure_code

,subject_id,icd_code,icd_version,long_title
0,10011398,3961,9,extracorporeal circulation auxiliary to open h...
1,10014729,3961,9,extracorporeal circulation auxiliary to open h...
2,10009628,3961,9,extracorporeal circulation auxiliary to open h...
3,10005817,3961,9,extracorporeal circulation auxiliary to open h...
4,10023771,3961,9,extracorporeal circulation auxiliary to open h...
...,...,...,...,...
717,10025463,0096,9,infusion of 4-factor prothrombin complex conce...
718,10002428,9390,9,non-invasive mechanical ventilation
719,10004733,3322,9,fiber-optic bronchoscopy
720,10004733,5523,9,closed [percutaneous] [needle] biopsy of kidney


In [118]:
j_procedure_code=j_procedure_code.drop(columns=['icd_code','icd_version'])

In [119]:
procedure_agg = j_procedure_code.groupby('subject_id').agg({
    'long_title': lambda x: list(set(x))
}).reset_index()

In [120]:
procedure_agg.rename(columns={'long_title':'procedure_name'}, inplace=True)

In [121]:
procedure_agg

,subject_id,procedure_name
0,10000032,[percutaneous abdominal drainage]
1,10001217,[central venous catheter placement with guidan...
2,10001725,"[repair of rectocele with graft or prosthesis,..."
3,10002428,"[insertion of endotracheal tube, arterial cath..."
4,10002495,[fluoroscopy of multiple coronary arteries usi...
...,...,...
87,10038999,[introduction of nutritional substance into lo...
88,10039708,[introduction of nutritional substance into lo...
89,10039831,[repair of blood vessel with tissue patch graf...
90,10039997,"[other computer assisted surgery, clipping of ..."


In [122]:
output_d=output_events.merge(d_items,on=['itemid'], how='inner')

In [123]:
output_d

,subject_id,stay_id,itemid,value,valueuom,label,category
0,10002428,35479615,226583,600,ml,Rectal Tube,Output
1,10029291,35146796,226583,225,ml,Rectal Tube,Output
2,10029291,35146796,226583,100,ml,Rectal Tube,Output
3,10029291,35146796,226583,450,ml,Rectal Tube,Output
4,10002428,35479615,226583,200,ml,Rectal Tube,Output
...,...,...,...,...,...,...,...
9357,10031404,35544374,226612,220,ml,Pericardial,Drains
9358,10031404,35544374,226612,75,ml,Pericardial,Drains
9359,10038999,39711498,226612,30,ml,Pericardial,Drains
9360,10014354,38017367,226612,10,ml,Pericardial,Drains


In [124]:
#aggregating variables by subject_id

output_agg = output_d.groupby('subject_id').agg({
    'value': lambda x: list(set(x)),
    'valueuom': lambda x: list(set(x)),
    'label': lambda x: list(set(x)),
    'category': lambda x: list(set(x))
}).reset_index()

In [125]:
output_agg

,subject_id,value,valueuom,label,category
0,10000032,[175],[ml],[Void],[Output]
1,10001217,"[225, 450, 900, 100, 200, 650, 300, 400, 275, ...",[ml],"[Emesis, Void]",[Output]
2,10001725,"[160, 100, 170, 75, 300, 175, 15, 50, 150, 375...",[ml],[Foley],[Output]
3,10002428,"[0, 130, 260, 5, 6, 135, 7, 10, 140, 525, 270,...",[ml],"[Pre-Admission, Foley, Pigtail #1, Chest Tube ...","[Drains, Output]"
4,10002495,"[0, 135, 270, 400, 3600, 20, 150, 25, 30, 40, ...",[ml],"[GU Irrigant/Urine Volume Out, GU Irrigant Vol...",[Output]
...,...,...,...,...,...
95,10038999,"[130, 8, 10, 650, 140, 15, 400, 20, 150, 25, 3...",[ml],"[Pericardial, L Pleural #1, Foley]","[Drains, Output]"
96,10039708,"[0, 130, 5, 8, 10, 12, 140, 15, 16, 145, 18, 2...",[ml],"[TF Residual, Fecal Bag, Oral Gastric, Foley]",[Output]
97,10039831,"[260, 270, 400, 150, 30, 160, 40, 300, 45, 50,...",[ml],"[OR Urine, OR EBL, Foley]",[Output]
98,10039997,"[450, 200, 650, 75, 50, 85, 150, 3000, 125, 350]",[ml],"[OR EBL, Void, OR Urine, Foley]",[Output]


# Joining more datasets.

The datasets had to be joined and cleaned before I could join them to the rest of the data

In [126]:
# Demographs (Combination of ICU stays, Patients, admin)

# Can then add another variable which is demograhics (Age, Status, language, race, Insurance)

# Demo_diag (Demographics+Diagnoses)
# Diagnoses + First Care + LOS
# Diagnoses + First Care + Death_Flag

# Demo_dipro (Demographics+Diagnones+Procedure)
# Diagnoses + Procedure + LOS
# Diagnoses + Procedure + Death_Flag

# Demo_out (Demographics+Output)
# Item_ID + Value + LOS
# Item_ID + Value + Death_Flag

#Demo_Diout (Demographics+Diagnoses+Output)
# Item_ID + Value + Diagnosis + LOS
# Item_ID + Value + Diagnosis + Death_flag

#Demo_care
#Prescription + Diagnosis + Procedure + LOS
#Prescription + Diagnosis + Procedure + Diagnosis

In [127]:
patient_admin=patients.merge(adm_agg,on=['subject_id'],how='inner')

In [128]:
patient_admin

,subject_id,gender,anchor_age,anchor_year_group,death_flag,insurance,language,marital_status,race
0,10014729,F,21,2011 - 2013,0,[other],[english],[single],[white - other european]
1,10003400,F,72,2011 - 2013,1,[medicare],[english],[married],[black/african american]
2,10002428,F,80,2011 - 2013,0,[medicare],[english],[widowed],[white]
3,10032725,F,38,2011 - 2013,1,[other],[english],[single],[black/african american]
4,10027445,F,48,2011 - 2013,1,[other],[english],[widowed],[white]
...,...,...,...,...,...,...,...,...,...
95,10004733,M,51,2014 - 2016,0,[medicaid],[english],[single],[unknown]
96,10021118,M,62,2014 - 2016,0,[other],[english],[married],[white]
97,10018501,M,83,2014 - 2016,0,[medicare],[english],[nan],[white]
98,10007058,M,48,2014 - 2016,0,[other],[english],[married],[white]


In [129]:
demographics=icu_stays.merge(patient_admin,on=['subject_id'], how='inner')

In [130]:
demographics

,subject_id,stay_id,first_careunit,los,gender,anchor_age,anchor_year_group,death_flag,insurance,language,marital_status,race
0,10018328,31269608,neuro stepdown,7.702512,F,83,2014 - 2016,0,[other],[english],[divorced],[white]
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran]
2,10020187,32554129,neuro intermediate,0.872685,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran]
3,10012853,31338022,trauma sicu (tsicu),3.766725,F,91,2014 - 2016,0,"[medicare, other]",[english],[widowed],[black/african american]
4,10020740,32145159,trauma sicu (tsicu),1.037106,M,56,2014 - 2016,0,[other],[english],[single],[white]
...,...,...,...,...,...,...,...,...,...,...,...,...
135,10023239,35024147,medical/surgical intensive care unit (micu/sicu),2.432998,F,29,2014 - 2016,0,[other],[english],"[married, single]",[white]
136,10001725,31205490,medical/surgical intensive care unit (micu/sicu),1.338588,F,46,2011 - 2013,0,[other],[english],[married],[white]
137,10035631,30932571,medical/surgical intensive care unit (micu/sicu),10.494259,M,63,2011 - 2013,1,[other],[english],[married],"[white, unknown]"
138,10020786,33683112,medical/surgical intensive care unit (micu/sicu),1.424757,F,86,2014 - 2016,0,[medicare],[english],[widowed],[white]


In [131]:
demo_diag=demographics.merge(diag_agg,on=['subject_id'],how='inner')

In [132]:
demo_diag

,subject_id,stay_id,first_careunit,los,gender,anchor_age,anchor_year_group,death_flag,insurance,language,marital_status,race,diagnoses_name
0,10018328,31269608,neuro stepdown,7.702512,F,83,2014 - 2016,0,[other],[english],[divorced],[white],"[supermarket, store or market as the place of ..."
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[presence of right artificial knee joint, pure..."
2,10020187,32554129,neuro intermediate,0.872685,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[presence of right artificial knee joint, pure..."
3,10012853,31338022,trauma sicu (tsicu),3.766725,F,91,2014 - 2016,0,"[medicare, other]",[english],[widowed],[black/african american],"[long-term (current) use of anticoagulants, co..."
4,10020740,32145159,trauma sicu (tsicu),1.037106,M,56,2014 - 2016,0,[other],[english],[single],[white],[pneumonitis due to inhalation of food or vomi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,10023239,35024147,medical/surgical intensive care unit (micu/sicu),2.432998,F,29,2014 - 2016,0,[other],[english],"[married, single]",[white],"[mechanical complication due to insulin pump, ..."
136,10001725,31205490,medical/surgical intensive care unit (micu/sicu),1.338588,F,46,2011 - 2013,0,[other],[english],[married],[white],"[vaginal enterocele, congenital or acquired, m..."
137,10035631,30932571,medical/surgical intensive care unit (micu/sicu),10.494259,M,63,2011 - 2013,1,[other],[english],[married],"[white, unknown]","[diarrhea, hepatitis, unspecified, graft-versu..."
138,10020786,33683112,medical/surgical intensive care unit (micu/sicu),1.424757,F,86,2014 - 2016,0,[medicare],[english],[widowed],[white],"[unspecified atrial fibrillation, chronic obst..."


In [133]:
demo_dipro=demo_diag.merge(procedure_agg,on=['subject_id'], how='inner')

In [134]:
demo_dipro

,subject_id,stay_id,first_careunit,los,gender,anchor_age,anchor_year_group,death_flag,insurance,language,marital_status,race,diagnoses_name,procedure_name
0,10018328,31269608,neuro stepdown,7.702512,F,83,2014 - 2016,0,[other],[english],[divorced],[white],"[supermarket, store or market as the place of ...",[restriction of intracranial artery with intra...
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[presence of right artificial knee joint, pure...",[restriction of intracranial artery with intra...
2,10020187,32554129,neuro intermediate,0.872685,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[presence of right artificial knee joint, pure...",[restriction of intracranial artery with intra...
3,10012853,31338022,trauma sicu (tsicu),3.766725,F,91,2014 - 2016,0,"[medicare, other]",[english],[widowed],[black/african american],"[long-term (current) use of anticoagulants, co...","[assistance with respiratory ventilation, grea..."
4,10020740,32145159,trauma sicu (tsicu),1.037106,M,56,2014 - 2016,0,[other],[english],[single],[white],[pneumonitis due to inhalation of food or vomi...,"[laparoscopic partial cholecystectomy, inserti..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,10023239,33846653,medical/surgical intensive care unit (micu/sicu),2.825370,F,29,2014 - 2016,0,[other],[english],"[married, single]",[white],"[mechanical complication due to insulin pump, ...","[excision of right middle lung lobe, via natur..."
127,10023239,35024147,medical/surgical intensive care unit (micu/sicu),2.432998,F,29,2014 - 2016,0,[other],[english],"[married, single]",[white],"[mechanical complication due to insulin pump, ...","[excision of right middle lung lobe, via natur..."
128,10001725,31205490,medical/surgical intensive care unit (micu/sicu),1.338588,F,46,2011 - 2013,0,[other],[english],[married],[white],"[vaginal enterocele, congenital or acquired, m...","[repair of rectocele with graft or prosthesis,..."
129,10035631,30932571,medical/surgical intensive care unit (micu/sicu),10.494259,M,63,2011 - 2013,1,[other],[english],[married],"[white, unknown]","[diarrhea, hepatitis, unspecified, graft-versu...","[transplant from live related donor, biopsy of..."


In [135]:
demo_out=demographics.merge(output_agg,on=['subject_id'],how='inner')

In [136]:
demo_out

,subject_id,stay_id,first_careunit,los,gender,anchor_age,anchor_year_group,death_flag,insurance,language,marital_status,race,value,valueuom,label,category
0,10018328,31269608,neuro stepdown,7.702512,F,83,2014 - 2016,0,[other],[english],[divorced],[white],"[0, 1, 130, 650, 140, 400, 150, 25, 30, 160, 3...",[ml],"[Void, Foley]",[Output]
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[0, 900, 400, 1300, 150, 30, 160, 800, 550, 30...",[ml],"[PACU Urine, Void, OR Urine, Foley]",[Output]
2,10020187,32554129,neuro intermediate,0.872685,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[0, 900, 400, 1300, 150, 30, 160, 800, 550, 30...",[ml],"[PACU Urine, Void, OR Urine, Foley]",[Output]
3,10012853,31338022,trauma sicu (tsicu),3.766725,F,91,2014 - 2016,0,"[medicare, other]",[english],[widowed],[black/african american],"[260, 135, 15, 20, 150, 25, 30, 35, 40, 300, 4...",[ml],"[Pre-Admission, Foley]",[Output]
4,10020740,32145159,trauma sicu (tsicu),1.037106,M,56,2014 - 2016,0,[other],[english],[single],[white],"[0, 130, 5, 265, 10, 400, 275, 20, 150, 280, 2...",[ml],"[Void, Emesis, Jackson Pratt #1, Oral Gastric,...","[Drains, Output]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,10023239,35024147,medical/surgical intensive care unit (micu/sicu),2.432998,F,29,2014 - 2016,0,[other],[english],"[married, single]",[white],"[0, 800, 900, 1000, 1100, 850, 600, 700]",[ml],[Void],[Output]
136,10001725,31205490,medical/surgical intensive care unit (micu/sicu),1.338588,F,46,2011 - 2013,0,[other],[english],[married],[white],"[160, 100, 170, 75, 300, 175, 15, 50, 150, 375...",[ml],[Foley],[Output]
137,10035631,30932571,medical/surgical intensive care unit (micu/sicu),10.494259,M,63,2011 - 2013,1,[other],[english],[married],"[white, unknown]","[0, 130, 2, 390, 10, 140, 15, 20, 150, 25, 155...",[ml],"[Condom Cath, TF Residual, Void, Foley]",[Output]
138,10020786,33683112,medical/surgical intensive care unit (micu/sicu),1.424757,F,86,2014 - 2016,0,[medicare],[english],[widowed],[white],"[200, 25, 400, 150]",[ml],"[Pre-Admission, Void]",[Output]


In [137]:
demo_diout=demo_out.merge(diag_agg,on=['subject_id'], how='inner')

In [138]:
demo_diout

,subject_id,stay_id,first_careunit,los,gender,anchor_age,anchor_year_group,death_flag,insurance,language,marital_status,race,value,valueuom,label,category,diagnoses_name
0,10018328,31269608,neuro stepdown,7.702512,F,83,2014 - 2016,0,[other],[english],[divorced],[white],"[0, 1, 130, 650, 140, 400, 150, 25, 30, 160, 3...",[ml],"[Void, Foley]",[Output],"[supermarket, store or market as the place of ..."
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[0, 900, 400, 1300, 150, 30, 160, 800, 550, 30...",[ml],"[PACU Urine, Void, OR Urine, Foley]",[Output],"[presence of right artificial knee joint, pure..."
2,10020187,32554129,neuro intermediate,0.872685,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[0, 900, 400, 1300, 150, 30, 160, 800, 550, 30...",[ml],"[PACU Urine, Void, OR Urine, Foley]",[Output],"[presence of right artificial knee joint, pure..."
3,10012853,31338022,trauma sicu (tsicu),3.766725,F,91,2014 - 2016,0,"[medicare, other]",[english],[widowed],[black/african american],"[260, 135, 15, 20, 150, 25, 30, 35, 40, 300, 4...",[ml],"[Pre-Admission, Foley]",[Output],"[long-term (current) use of anticoagulants, co..."
4,10020740,32145159,trauma sicu (tsicu),1.037106,M,56,2014 - 2016,0,[other],[english],[single],[white],"[0, 130, 5, 265, 10, 400, 275, 20, 150, 280, 2...",[ml],"[Void, Emesis, Jackson Pratt #1, Oral Gastric,...","[Drains, Output]",[pneumonitis due to inhalation of food or vomi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,10023239,35024147,medical/surgical intensive care unit (micu/sicu),2.432998,F,29,2014 - 2016,0,[other],[english],"[married, single]",[white],"[0, 800, 900, 1000, 1100, 850, 600, 700]",[ml],[Void],[Output],"[mechanical complication due to insulin pump, ..."
136,10001725,31205490,medical/surgical intensive care unit (micu/sicu),1.338588,F,46,2011 - 2013,0,[other],[english],[married],[white],"[160, 100, 170, 75, 300, 175, 15, 50, 150, 375...",[ml],[Foley],[Output],"[vaginal enterocele, congenital or acquired, m..."
137,10035631,30932571,medical/surgical intensive care unit (micu/sicu),10.494259,M,63,2011 - 2013,1,[other],[english],[married],"[white, unknown]","[0, 130, 2, 390, 10, 140, 15, 20, 150, 25, 155...",[ml],"[Condom Cath, TF Residual, Void, Foley]",[Output],"[diarrhea, hepatitis, unspecified, graft-versu..."
138,10020786,33683112,medical/surgical intensive care unit (micu/sicu),1.424757,F,86,2014 - 2016,0,[medicare],[english],[widowed],[white],"[200, 25, 400, 150]",[ml],"[Pre-Admission, Void]",[Output],"[unspecified atrial fibrillation, chronic obst..."


In [139]:
demo_care=demo_dipro.merge(presc_agg,on=['subject_id'], how='inner')

In [140]:
demo_care

,subject_id,stay_id,first_careunit,los,gender,anchor_age,anchor_year_group,death_flag,insurance,language,marital_status,race,diagnoses_name,procedure_name,drug
0,10018328,31269608,neuro stepdown,7.702512,F,83,2014 - 2016,0,[other],[english],[divorced],[white],"[supermarket, store or market as the place of ...",[restriction of intracranial artery with intra...,"[vitamin d, levetiracetam, pravastatin, iso-os..."
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[presence of right artificial knee joint, pure...",[restriction of intracranial artery with intra...,"[levetiracetam, iso-osmotic sodium chloride, h..."
2,10020187,32554129,neuro intermediate,0.872685,F,63,2014 - 2016,0,[other],[?],[married],[hispanic/latino - salvadoran],"[presence of right artificial knee joint, pure...",[restriction of intracranial artery with intra...,"[levetiracetam, iso-osmotic sodium chloride, h..."
3,10012853,31338022,trauma sicu (tsicu),3.766725,F,91,2014 - 2016,0,"[medicare, other]",[english],[widowed],[black/african american],"[long-term (current) use of anticoagulants, co...","[assistance with respiratory ventilation, grea...","[levothyroxine sodium, vitamin d, multivitamin..."
4,10020740,32145159,trauma sicu (tsicu),1.037106,M,56,2014 - 2016,0,[other],[english],[single],[white],[pneumonitis due to inhalation of food or vomi...,"[laparoscopic partial cholecystectomy, inserti...","[chlorhexidine gluconate 0.12% oral rinse, pot..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,10023239,33846653,medical/surgical intensive care unit (micu/sicu),2.825370,F,29,2014 - 2016,0,[other],[english],"[married, single]",[white],"[mechanical complication due to insulin pump, ...","[excision of right middle lung lobe, via natur...","[levothyroxine sodium, insulin human regular, ..."
127,10023239,35024147,medical/surgical intensive care unit (micu/sicu),2.432998,F,29,2014 - 2016,0,[other],[english],"[married, single]",[white],"[mechanical complication due to insulin pump, ...","[excision of right middle lung lobe, via natur...","[levothyroxine sodium, insulin human regular, ..."
128,10001725,31205490,medical/surgical intensive care unit (micu/sicu),1.338588,F,46,2011 - 2013,0,[other],[english],[married],[white],"[vaginal enterocele, congenital or acquired, m...","[repair of rectocele with graft or prosthesis,...","[metolazone, iso-osmotic sodium chloride, albu..."
129,10035631,30932571,medical/surgical intensive care unit (micu/sicu),10.494259,M,63,2011 - 2013,1,[other],[english],[married],"[white, unknown]","[diarrhea, hepatitis, unspecified, graft-versu...","[transplant from live related donor, biopsy of...","[chlorhexidine gluconate 0.12% oral rinse, vit..."


In [141]:
# Is the patient eldery
demo_care['is_elderly']= demo_care['anchor_age'] >= 65

# Length of stay in hours (Slightly more precise)
demo_care['los_hours']=demo_care['los']*24

# Did the patient have a long stay. Anything over a week
demo_care['long_stay_flag']=demo_care['los']>7

# Did a patient die within 24 hours?
demo_care['death_within_24h_flag'] = ((demo_care['los_hours'] <= 24) & (demo_care['death_flag'] == 1)).astype(int)

# Does the patient have diabetes?
demo_care['has_diabetes']=demo_care['diagnoses_name'].apply(
    lambda x: any('diabetes' in d.lower() for d in x) if isinstance(x, list) else False)

# Does the patient have any heart issues
demo_care['has_cardiac_issues']=demo_care['diagnoses_name'].apply(
    lambda x: any(any(k in d.lower() for k in ['heart','cardiac','coronary'])for d in x)if isinstance(x, list) else False)

# Diagnoses per patient
demo_care['num_diagnoses'] = demo_care['diagnoses_name'].apply(lambda x: len(set(x)) if isinstance(x, list) else 0)
# Procedures per patient
demo_care['num_procedures'] = demo_care['procedure_name'].apply(lambda x: len(set(x)) if isinstance(x, list) else 0)
# Drugs per patient
demo_care['num_drugs'] = demo_care['drug'].apply(
    lambda x: len(set([d.lower().strip() for d in x])) if isinstance(x, list) else 0
)



# Was the patient readmitted to the hospital
readmission_flag=( demo_care.groupby('subject_id')['stay_id']
                 .transform('count')>1)

demo_care['readmission_flag']=readmission_flag

# encoding ICU Type
le= LabelEncoder()
demo_care['icu_type_encoded']=le.fit_transform(demo_care['first_careunit'].astype(str))

#encoding gender
demo_care['gender']=le.fit_transform(demo_care['gender'].astype(str))

# Encoding anchor_year_group
demo_care['anchor_year_group']=le.fit_transform(demo_care['anchor_year_group'].astype(str))

#encoding language
demo_care['language']=le.fit_transform(demo_care['language'].astype(str))

# Encoding insurance
demo_care['insurance']=le.fit_transform(demo_care['insurance'].astype(str))

#Encoding Readmission Flag
demo_care['readmission_flag']=le.fit_transform(demo_care['readmission_flag'].astype(str))

#Encoding has_cardiac_issue
demo_care['has_cardiac_issues']=le.fit_transform(demo_care['has_cardiac_issues'].astype(str))

#encoding race
demo_care['race']=le.fit_transform(demo_care['race'].astype(str))

#encoding marital_status
demo_care['marital_status']=le.fit_transform(demo_care['marital_status'].astype(str))

# Mortality rate by ICU unit
mortality_rate=( demo_care.groupby('first_careunit')['death_flag']
               .mean()
               .to_dict())

demo_care['mortality_rate_by_unit']=demo_care['first_careunit'].map(mortality_rate)

# Cap age 
demo_care['age_capped'] = np.where(demo_care['anchor_age'] > 89, 90, demo_care['anchor_age'])

#aggregated age
bins = [21, 40, 65, 80, 90, np.inf]
labels = ['young_adult', 'adult', 'senior', 'elderly', '90_plus']

demo_care['age_group'] = pd.cut(
    demo_care['age_capped'],
    bins=bins,
    labels=labels,
    right=False,       
    include_lowest=True
)

#encoding age_group
demo_care['age_group']=le.fit_transform(demo_care['age_group'].astype(str))

# Reduces right skew in LOS
demo_care['log_los'] = np.log1p(demo_care['los'])


In [142]:
demo_care

,subject_id,stay_id,first_careunit,los,gender,anchor_age,anchor_year_group,death_flag,insurance,language,...,has_cardiac_issues,num_diagnoses,num_procedures,num_drugs,readmission_flag,icu_type_encoded,mortality_rate_by_unit,age_capped,age_group,log_los
0,10018328,31269608,neuro stepdown,7.702512,0,83,1,0,4,1,...,1,25,2,48,0,5,0.000000,83,2,2.163612
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662,0,63,1,0,4,0,...,1,12,1,30,1,6,0.333333,63,1,1.864493
2,10020187,32554129,neuro intermediate,0.872685,0,63,1,0,4,0,...,1,12,1,30,1,4,0.000000,63,1,0.627373
3,10012853,31338022,trauma sicu (tsicu),3.766725,0,91,1,0,1,1,...,1,66,1,52,0,8,0.312500,90,0,1.561659
4,10020740,32145159,trauma sicu (tsicu),1.037106,1,56,1,0,4,1,...,1,44,13,82,1,8,0.312500,56,1,0.711530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,10023239,33846653,medical/surgical intensive care unit (micu/sicu),2.825370,0,29,1,0,4,1,...,0,29,3,50,1,3,0.400000,29,4,1.341655
127,10023239,35024147,medical/surgical intensive care unit (micu/sicu),2.432998,0,29,1,0,4,1,...,0,29,3,50,1,3,0.400000,29,4,1.233434
128,10001725,31205490,medical/surgical intensive care unit (micu/sicu),1.338588,0,46,0,0,4,1,...,0,18,3,35,0,3,0.400000,46,1,0.849547
129,10035631,30932571,medical/surgical intensive care unit (micu/sicu),10.494259,1,63,0,1,4,1,...,1,101,21,176,0,3,0.400000,63,1,2.441848


In [143]:
demo_care.isna().sum()

subject_id                0
stay_id                   0
first_careunit            0
los                       0
gender                    0
anchor_age                0
anchor_year_group         0
death_flag                0
insurance                 0
language                  0
marital_status            0
race                      0
diagnoses_name            0
procedure_name            0
drug                      0
is_elderly                0
los_hours                 0
long_stay_flag            0
death_within_24h_flag     0
has_diabetes              0
has_cardiac_issues        0
num_diagnoses             0
num_procedures            0
num_drugs                 0
readmission_flag          0
icu_type_encoded          0
mortality_rate_by_unit    0
age_capped                0
age_group                 0
log_los                   0
dtype: int64

In [144]:
demo_care

,subject_id,stay_id,first_careunit,los,gender,anchor_age,anchor_year_group,death_flag,insurance,language,...,has_cardiac_issues,num_diagnoses,num_procedures,num_drugs,readmission_flag,icu_type_encoded,mortality_rate_by_unit,age_capped,age_group,log_los
0,10018328,31269608,neuro stepdown,7.702512,0,83,1,0,4,1,...,1,25,2,48,0,5,0.000000,83,2,2.163612
1,10020187,37509585,neuro surgical intensive care unit (neuro sicu),5.452662,0,63,1,0,4,0,...,1,12,1,30,1,6,0.333333,63,1,1.864493
2,10020187,32554129,neuro intermediate,0.872685,0,63,1,0,4,0,...,1,12,1,30,1,4,0.000000,63,1,0.627373
3,10012853,31338022,trauma sicu (tsicu),3.766725,0,91,1,0,1,1,...,1,66,1,52,0,8,0.312500,90,0,1.561659
4,10020740,32145159,trauma sicu (tsicu),1.037106,1,56,1,0,4,1,...,1,44,13,82,1,8,0.312500,56,1,0.711530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,10023239,33846653,medical/surgical intensive care unit (micu/sicu),2.825370,0,29,1,0,4,1,...,0,29,3,50,1,3,0.400000,29,4,1.341655
127,10023239,35024147,medical/surgical intensive care unit (micu/sicu),2.432998,0,29,1,0,4,1,...,0,29,3,50,1,3,0.400000,29,4,1.233434
128,10001725,31205490,medical/surgical intensive care unit (micu/sicu),1.338588,0,46,0,0,4,1,...,0,18,3,35,0,3,0.400000,46,1,0.849547
129,10035631,30932571,medical/surgical intensive care unit (micu/sicu),10.494259,1,63,0,1,4,1,...,1,101,21,176,0,3,0.400000,63,1,2.441848


In [145]:
cols_to_drop=['subject_id','stay_id','first_careunit','anchor_age','diagnoses_name','procedure_name','drug']

demo_care_clean=demo_care.drop(columns=cols_to_drop,errors='ignore')


In [148]:
demo_care_clean

,los,gender,anchor_year_group,death_flag,insurance,language,marital_status,race,is_elderly,los_hours,...,has_cardiac_issues,num_diagnoses,num_procedures,num_drugs,readmission_flag,icu_type_encoded,mortality_rate_by_unit,age_capped,age_group,log_los
0,7.702512,0,1,0,4,1,0,15,True,184.860278,...,1,25,2,48,0,5,0.000000,83,2,2.163612
1,5.452662,0,1,0,4,0,4,5,False,130.863889,...,1,12,1,30,1,6,0.333333,63,1,1.864493
2,0.872685,0,1,0,4,0,4,5,False,20.944444,...,1,12,1,30,1,4,0.000000,63,1,0.627373
3,3.766725,0,1,0,1,1,8,0,True,90.401389,...,1,66,1,52,0,8,0.312500,90,0,1.561659
4,1.037106,1,1,0,4,1,6,15,False,24.890556,...,1,44,13,82,1,8,0.312500,56,1,0.711530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,2.825370,0,1,0,4,1,2,15,False,67.808889,...,0,29,3,50,1,3,0.400000,29,4,1.341655
127,2.432998,0,1,0,4,1,2,15,False,58.391944,...,0,29,3,50,1,3,0.400000,29,4,1.233434
128,1.338588,0,0,0,4,1,4,15,False,32.126111,...,0,18,3,35,0,3,0.400000,46,1,0.849547
129,10.494259,1,0,1,4,1,4,14,False,251.862222,...,1,101,21,176,0,3,0.400000,63,1,2.441848


In [150]:
train_df,temp_df=train_test_split(demo_care_clean,
                                 test_size=0.4,
                                 stratify=demo_care['death_flag'],
                                 random_state=42)

val_df, test_df=train_test_split(temp_df,
                                test_size=0.5,
                                stratify=temp_df['death_flag'],
                                random_state=42)

#check the shape
print(f"Train: {train_df.shape}")
print(f"Validation: {val_df.shape}")
print(f"Test: {test_df.shape}")

# Optional: confirm stratification
print(train_df['death_flag'].value_counts(normalize=True))
print(val_df['death_flag'].value_counts(normalize=True))
print(test_df['death_flag'].value_counts(normalize=True))

Train: (78, 23)
Validation: (26, 23)
Test: (27, 23)
death_flag
0    0.653846
1    0.346154
Name: proportion, dtype: float64
death_flag
0    0.653846
1    0.346154
Name: proportion, dtype: float64
death_flag
0    0.666667
1    0.333333
Name: proportion, dtype: float64


In [151]:
# creating folder for exported data
final_data_path = os.path.join("..", "Data", "Final Data")
os.makedirs(final_data_path, exist_ok=True)

In [153]:
# Export dataset
demo_care.to_csv(os.path.join(final_data_path, "demo_care.csv"), index=False)
demo_care_clean.to_csv(os.path.join(final_data_path, "demo_care_clean.csv"), index=False)
train_df.to_csv(os.path.join(final_data_path, "train_dataset.csv"), index=False)
val_df.to_csv(os.path.join(final_data_path, "validation_dataset.csv"), index=False)
test_df.to_csv(os.path.join(final_data_path, "test_dataset.csv"), index=False)
